In [11]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException, WebDriverException, ElementClickInterceptedException
import re
from bs4 import BeautifulSoup
from time import sleep
import os

In [2]:
def get_data_event(event):
    cleaned_event = re.sub('<tbody>|<tr>|<td>|</tbody>|</tr>|</td>|<td colspan="2">|\t','', event)
    nsymbols_data = 16
    date = cleaned_event[:nsymbols_data]
    cleaned_event_ = cleaned_event[nsymbols_data:].strip()
    splitted_event = cleaned_event_.split(';')
    players = splitted_event[0]
    cleaned_event_ = splitted_event[1].strip()
    if 'Победа' in cleaned_event:
        winner = cleaned_event_.split(',Тотал')[0].split(',  ')[1]
        splitted_event = cleaned_event_.split(', ')[0].split('  ')
        event_ = ' '.join(splitted_event[1:3])
        score = cleaned_event_.split('Итог матча:    ')[1].split(',  ')[0]
    else:
        winner = ''
        event_ = ' '.join(cleaned_event_.split('  ')[1:3])
        score = ''
    return date, event_, players, winner, score


def get_data_table(tables):
    page_tables = []
    for str_table in tables:
        soup = BeautifulSoup(str_table, 'html.parser')
        cleaned_table_data = [
            re.sub('<tr> | </tr>| class="c1"| class="c2"', '', str(x)) for x in soup.find_all('tr')[1:-1]
        ]
        columns = soup.find_all('th')
        columns = [re.sub('<th> | </th>', '', str(x)) for x in columns]
        table = []
        for str_ in cleaned_table_data: 
            soup_str = BeautifulSoup(str_, 'html.parser')
            table.append([x.text for x in soup_str.find_all('td')])
        table = pd.DataFrame(data=table, columns=columns)
        page_tables.append(table)
    return page_tables

In [3]:
# correct one-date parser

# url = 'https://line4bet.ru/1x-13-03-2023-football/'
# driver = webdriver.Chrome()
# driver.get(url)
# assert 'Движение линий' in driver.title
# # "Теннис" button searching
# tennis_btn = driver.find_elements(By.ID, 'tennis_btn')
# assert len(tennis_btn) > 0
# tennis_btn[0].click()


# # looking for available date
# previous_date = dt.datetime.today() - dt.timedelta(days=1)
# prev_day = previous_date.day
# dates_objects = driver.find_elements(By.XPATH, "//td[@Data-handler]")
# try:
#     dates_objects[prev_day-1].click()
# except:
#     dates_objects = driver.find_elements(By.XPATH, "//td[@Data-handler]")
#     dates_objects[prev_day-1].click()


# bcs = ['tab_1x', 'tab_bc', 'tab_fb', 'tab_mf']
# data_events = {bc: [] for bc in bcs}
# data_tables = {bc: [] for bc in bcs}
# for bc in bcs:
#     sleep(1)
#     driver.find_elements(By.ID, bc)[0].click()
#     sleep(1)
#     npages = driver.find_elements(By.CLASS_NAME, 'pages')
#     # pages iterating
#     for i in range(len(npages)):
#         try:
#             driver.execute_script("arguments[0].click();", npages[i])
#         except (StaleElementReferenceException, WebDriverException):
#             npages = driver.find_elements(By.CLASS_NAME, 'pages')
#             driver.execute_script("arguments[0].click();", npages[i])    
#         sleep(1)
#         html_source_code = driver.execute_script("return document.body.innerHTML;")
#         html_soup = BeautifulSoup(html_source_code, 'html.parser')
#         tbody_data = np.array([str(x) for x in html_soup.find_all(["tbody"])], dtype='object')
#         # events parsing
#         events_indexes = ['<tbody><tr><td>' in x and 'colspan="2"' in x for x in tbody_data]
#         events = tbody_data[events_indexes]
#         # data_events.append([get_data_event(x) for x in events])
#         data_events[bc].append([get_data_event(x) for x in events])
#         # tables parsing
#         tables = tbody_data[['Дата-время скана линии' in x for x in tbody_data]]
#         # data_tables.append(get_data_table(tables))
#         data_tables[bc].append(get_data_table(tables))

# Calendar switcher

In [3]:
def save_data(i, current_page_month, current_page_year, data_tables, data_events):
    data_folder = '../data/'
    for bc in data_events.keys():
        # events saving
        events_table = pd.DataFrame(
            np.vstack(data_events[bc]), 
            columns=['date', 'event', 'players', 'winner', 'score']
        )
        bc_folder = bc.split('_')[1]
        file_name = f'{current_page_year}_{current_page_month.lower()}_{i+1}'
        events_table.to_pickle(data_folder+'/'+bc_folder+'/events/'+file_name+'.pkl')

        # tables saving
        path = data_folder+'/'+bc_folder+'/tables/'+file_name
        if not os.path.exists(path):
            os.makedirs(path)
        try:
            tables = np.hstack(np.array(data_tables[bc], dtype='object'))
        except ValueError:
            for table_list in data_tables[bc]:
                for table in table_list:
                    table.to_pickle(path+'/'+str(j)+'.pkl')
        else:
            for j, table in enumerate(tables):
                table.to_pickle(path+'/'+str(j)+'.pkl')
        

In [4]:
url = 'https://line4bet.ru/1x-13-03-2023-football/'
driver = webdriver.Chrome()
driver.get(url)
assert 'Движение линий' in driver.title
# "Теннис" button searching
tennis_btn = driver.find_elements(By.ID, 'tennis_btn')
assert len(tennis_btn) > 0
tennis_btn[0].click()

In [5]:
import warnings
warnings.filterwarnings("ignore")

In [49]:
# previous_date = dt.datetime.today() - dt.timedelta(days=1)
# prev_day = previous_date.day

bcs = ['tab_1x', 'tab_bc', 'tab_fb', 'tab_mf']
current_page_month = driver.find_elements(By.CLASS_NAME, 'ui-datepicker-month')[0].text
current_page_year = driver.find_elements(By.CLASS_NAME, 'ui-datepicker-year')[0].text
# first_month_flg = True
first_month_flg = False
while current_page_month != 'Декабрь' or current_page_year != '2019':
    dates_objects = driver.find_elements(By.XPATH, "//td[@Data-handler]")
    n_range = range(len(dates_objects)-1, -1, -1)
    # Date switcher
    for i in n_range:
        # skip first pages_without data
        if first_month_flg and (i+1 > prev_day):
            continue
        dates_objects = driver.find_elements(By.XPATH, "//td[@Data-handler]")
        try:
            dates_objects[i].click()
        except:
            dates_objects = driver.find_elements(By.XPATH, "//td[@Data-handler]")
            dates_objects[i].click()
        # objects with data for current date
        data_events = {bc: [] for bc in bcs}
        data_tables = {bc: [] for bc in bcs}
        # BC switcher
        for bc in bcs:
            sleep(1)
            try:
                driver.find_elements(By.ID, bc)[0].click()
            except ElementClickInterceptedException:
                element = driver.find_elements(By.ID, bc)[0]
                driver.execute_script("arguments[0].click();", element)
            sleep(1)
            npages = driver.find_elements(By.CLASS_NAME, 'pages')
            len_pages = len(npages)
            if not len_pages:
                len_pages = 1
            # pages iterating
            for j in range(len_pages):
                if len_pages > 1:
                    try:
                        driver.execute_script("arguments[0].click();", npages[j])
                    except (StaleElementReferenceException, WebDriverException):
                        npages = driver.find_elements(By.CLASS_NAME, 'pages')
                        driver.execute_script("arguments[0].click();", npages[j])    
                sleep(1)
                html_source_code = driver.execute_script("return document.body.innerHTML;")
                html_soup = BeautifulSoup(html_source_code, 'html.parser')
                tbody_data = np.array([str(x) for x in html_soup.find_all(["tbody"])], dtype='object')
                # events parsing
                events_indexes = ['<tbody><tr><td>' in x and 'colspan="2"' in x for x in tbody_data]
                events = tbody_data[events_indexes]
                # data_events.append([get_data_event(x) for x in events])
                data_events[bc].append([get_data_event(x) for x in events])
                # tables parsing
                tables = tbody_data[['Дата-время скана линии' in x for x in tbody_data]]
                # data_tables.append(get_data_table(tables))
                data_tables[bc].append(get_data_table(tables))
        save_data(i, current_page_month, current_page_year, data_tables, data_events)
    first_month_flg = False
    # month switcher
    left_triangle = driver.find_elements(By.LINK_TEXT, 'Назад')
    left_triangle[0].click()
    current_page_month = driver.find_elements(By.CLASS_NAME, 'ui-datepicker-month')[0].text
    current_page_year = driver.find_elements(By.CLASS_NAME, 'ui-datepicker-year')[0].text

IndexError: list index out of range